# AutoEncoder: Mnist Digits

In [0]:
# Base Mnist Digits
from keras.datasets import mnist
import numpy as np
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
# Shape da base
print(x_train.shape)

# Quantidade de imagens da base
print(len(x_train))

# Quantidade de pixels de cada imagem
print(np.prod(x_train.shape[1:]))

In [0]:
# Normalização
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

In [0]:
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

# Tamanho da camada latente: 
encoding_dim = 32  # 32 dimensões  -> fator de compressão de 24.5

# Input
input_img = Input(shape=(784,))

# "encoded" é a representação codificada do nosso input
encoded = Dense(encoding_dim, activation='relu')(input_img)

# "decoded" é a reconstrução do input
decoded = Dense(784, activation='sigmoid')(encoded)

# "autoencoder" é o modelo que codificada o input e o reconstrói
autoencoder = Model(input_img, decoded)

# Compila a rede neural
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# Mostrar a estrtura do modelo criado
config = autoencoder.get_config()
model = Model.from_config(config)
model.summary()

In [0]:
# Treinamento do autoencoder
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

## Testar o autoencoder

In [0]:
# codificar e decodificar alguns digitos da base de teste
decoded_imgs = autoencoder.predict(x_test)

In [0]:
# Função para comparação do original com o reconstruido
import matplotlib.pyplot as plt

def plotTest(testDb, netDB):
  n = 10  # quantos digitos serão mostrados
  plt.figure(figsize=(20, 4))
  for i in range(n):
      # mostrar original
      ax = plt.subplot(2, n, i + 1)
      plt.imshow(testDb[i].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

      # mostrar reconstrução
      ax = plt.subplot(2, n, i + 1 + n)
      plt.imshow(netDB[i].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
  plt.show()

In [0]:
# Chamar a função que imprime o original e a reconstrução
plotTest(x_test, decoded_imgs)

# Exercícios

### Crie o modelo "encoder" que retorne a representação de uma imagem codificada


In [0]:
encoder = Model(input_img, encoded)

### Escolha duas imagens da base de teste e salve no objeto imagens

In [0]:
imagens = x_test[0:2,:]


### Plote a imagem

In [0]:
# Imagem 1
plt.imshow(imagens[0,:].reshape(28, 28))
plt.gray()

In [0]:
# Imagem 2
plt.imshow(imagens[1,:].reshape(28, 28))
plt.gray()

### Apresente as imagens selecionadas da base de teste e verifique a saída codificada das imagens

In [0]:
decoded_imgs = encoder.predict(imagens)
decoded_imgs

### Use a base codificada para treinar uma MLP para classificação de digitos

In [0]:
# Codificar a base de treino e teste
encoded_x_train = encoder.predict(x_train)
encoded_x_test = encoder.predict(x_test)
print(encoded_x_train.shape)
print(encoded_x_test.shape)


In [0]:
encoded_x_train.shape[1:]

In [0]:
from keras import Model, Sequential
from keras.layers import Dense
from keras.optimizers import SGD

# Estrtura da rede neural mlp
RN = Sequential()
RN.add(Dense(20, input_shape = encoded_x_train.shape[1:],activation = 'sigmoid'))
RN.add(Dense(10, activation = 'sigmoid'))

RN.compile(optimizer = 'sgd', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
RN.summary()

In [0]:
RN.fit(encoded_x_train, y_train, epochs = 100)

In [0]:
score = RN.evaluate(encoded_x_test, y_test, verbose = 0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

### Prever as duas imagens escolhidas no exercício anterior

In [0]:
RN.predict_classes(encoded_x_test[0:2,:])